In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.Category.value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [4]:
df['spam'] = df['Category'].apply(lambda x: 1 if x =='spam' else 0)


In [5]:
df.shape


(5572, 3)

In [6]:
df.head()


,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


Train test split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)

In [10]:
X_train.shape

(4457,)

In [11]:
X_test.shape


(1115,)

In [12]:
type(X_train)

pandas.core.series.Series

In [13]:
X_train[:4]

5487    2p per min to call Germany 08448350055 from yo...
2811                 Mm so you asked me not to call radio
5190    Our records indicate u maybe entitled to 5000 ...
3830             Sure, I'll see if I can come by in a bit
Name: Message, dtype: object

In [14]:
type(y_train)

pandas.core.series.Series

In [15]:
y_train[:4]

5487    1
2811    0
5190    1
3830    0
Name: spam, dtype: int64

In [16]:
type(X_train.values)

numpy.ndarray

# Create bag of words representation using CountVectorizer

In [17]:
# Create bag of words representation using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<4457x7670 sparse matrix of type '<class 'numpy.int64'>'
	with 59207 stored elements in Compressed Sparse Row format>

In [18]:
X_train_cv.toarray()[:2][0]


array([0, 0, 0, ..., 0, 0, 0])

In [19]:
X_train_cv.shape


(4457, 7670)

In [20]:
v.get_feature_names_out()[1771]


'chip'

In [21]:
v.vocabulary_


{'2p': 400,
 'per': 5093,
 'min': 4422,
 'to': 6869,
 'call': 1586,
 'germany': 3098,
 '08448350055': 60,
 'from': 3000,
 'your': 7634,
 'bt': 1518,
 'line': 4079,
 'just': 3835,
 'check': 1728,
 'planettalkinstant': 5176,
 'com': 1874,
 'for': 2917,
 'info': 3638,
 'text': 6725,
 'stop': 6438,
 'opt': 4912,
 'out': 4946,
 'mm': 4470,
 'so': 6227,
 'you': 7629,
 'asked': 1079,
 'me': 4352,
 'not': 4778,
 'radio': 5480,
 'our': 4944,
 'records': 5580,
 'indicate': 3630,
 'maybe': 4345,
 'entitled': 2594,
 '5000': 525,
 'pounds': 5283,
 'in': 3604,
 'compensation': 1897,
 'the': 6750,
 'accident': 756,
 'had': 3273,
 'claim': 1795,
 'free': 2965,
 'reply': 5658,
 'with': 7468,
 'this': 6788,
 'msg': 4559,
 'txt': 7024,
 'sure': 6563,
 'll': 4110,
 'see': 5935,
 'if': 3567,
 'can': 1611,
 'come': 1879,
 'by': 1569,
 'bit': 1351,
 'heart': 3356,
 'is': 3707,
 'empty': 2565,
 'without': 7471,
 'love': 4177,
 'mind': 4425,
 'wisdom': 7459,
 'eyes': 2715,
 'dreams': 2422,
 'amp': 931,
 'life'

In [22]:
X_train_np = X_train_cv.toarray()
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
np.where(X_train_np[0]!=0)


(array([  60,  400, 1518, 1586, 1728, 1874, 2917, 3000, 3098, 3638, 3835,
        4079, 4422, 4912, 4946, 5093, 5176, 6438, 6725, 6869, 7634]),)

In [26]:
# X_train[:4][1579]

In [25]:
X_train_np[0][1771]

np.int64(0)

# Train the naive bayes model

In [27]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

# In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook.
# On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.

In [28]:
X_test_cv = v.transform(X_test)


In [29]:
# Evaluate Performance
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       961
           1       0.98      0.90      0.94       154

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [31]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

In [33]:
# Train the model using sklearn pipeline and reduce number of lines of code
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [34]:
clf.fit(X_train, y_train)


Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [35]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       961
           1       0.98      0.90      0.94       154

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

